## Домашнее задание № 1. Spark RDD API
ФИО: Алёхин Арсений Михайлович

Группа: ИУ6-54Б

[Набор данных](../data/places.csv)

```python
# Наименование столбцов
['ID', 'Name', 'global_id', 'IsNetObject', 'OperatingCompany', 'TypeObject', 'AdmArea', 'District', 'Address', 'PublicPhone', 'SeatsCount', 'SocialPrivileges', 'Longitude_WGS84', 'Latitude_WGS84', 'geoData']
```

**Замечание**: 1) для парсинга данных используйте модуль `csv`; 2) для вычисления расстояния используйте формулу гаверсинуса.

1. Рассчитайте расстояние от заданной точки (`lat=55.751244`, `lng=37.618423`) до каждого заведения общепита из набора данных. Выведите первые 10.
2. Рассчитайте расстояние между всеми заведениями общепита из набора данных. Выведите первые 10.
3. Выведите топ-10 наиболее близких и наиболее отдаленных заведений.


### Импорт модулей и инициализация SparkContext

In [1]:
import csv
from math import radians, cos, sin, asin, sqrt
from pyspark import SparkContext, SparkConf
import os
conf = SparkConf().setAppName("hw1").set("spark.driver.host", "127.0.0.1").set("spark.driver.bindAddress", "127.0.0.1")

sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")


26/01/07 18:43:54 WARN Utils: Your hostname, MacBook-Air-Arsenij-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.139 instead (on interface en0)
26/01/07 18:43:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/07 18:43:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Функция гаверсинуса

In [3]:
def get_haversin_dist(lon1, lat1, lon2, lat2):
    """Расстояние между двумя точками на Земле в км"""
    lon1, lat1, lon2, lat2 = map(radians, [float(lon1), float(lat1), float(lon2), float(lat2)])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * asin(sqrt(a)) * 6371

# Тест
test_dist = get_haversin_dist(37.618423, 55.751244, 37.615739, 55.753594)
print(f"Красная площадь - Кремль: {test_dist:.2f} км")



Красная площадь - Кремль: 0.31 км


### Функция парсинга CSV

In [4]:
def parse_csv_line(line):
    """Парсит CSV строку"""
    return next(csv.reader([line]))


### Загрузка данных

In [6]:
input_path = "places.csv"
raw_rdd = sc.textFile(input_path)
header = raw_rdd.first()

print(f"Всего строк: {raw_rdd.count()}")
print(f"Партиции: {raw_rdd.getNumPartitions()}")
print("Нужные столбцы: [1] Name, [12] Longitude, [13] Latitude")


Всего строк: 769
Партиции: 2
Нужные столбцы: [1] Name, [12] Longitude, [13] Latitude


### Создание основного RDD

In [7]:
data_rdd = raw_rdd.filter(lambda x: x != header).map(parse_csv_line).cache()

total_records = data_rdd.count()

print(f"Всего записей: {total_records}")
print(f"Партиции: {data_rdd.getNumPartitions()}")


Всего записей: 768
Партиции: 2


### Задание 1
Рассчитайте расстояние от заданной точки (lat=55.751244, lng=37.618423) до каждого заведения общепита из набора данных. Выведите первые 10.

In [9]:
TARGET_LAT = 55.751244
TARGET_LNG = 37.618423


print(f"Красная площадь: LAT={TARGET_LAT}, LNG={TARGET_LNG}\n")

target_dist_rdd = data_rdd.map(lambda row: (
    row[1],
    get_haversin_dist(TARGET_LNG, TARGET_LAT, row[12], row[13])
))

results = target_dist_rdd.take(10)

print(f"{'Название':<50} {'Расстояние (км)':>10}")
print("-" * 60)
for name, dist in results:
    name_short = name[:50] if len(name) <= 50 else name[:47] + "..."
    print(f"{name_short:<50} {round(dist, 5):>10}")



Красная площадь: LAT=55.751244, LNG=37.618423

Название                                           Расстояние (км)
------------------------------------------------------------
МУ-МУ                                                 4.41145
КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА                  4.14157
Дом 12                                                2.97678
Чито-Ра                                               3.03484
Бар- буфет «Николай»                                  2.82873
Флорентини                                            2.88858
Beer Gik                                              1.56531
Погребок                                              1.56531
Пробка Гриль                                          1.64999
TEMPO DI PASTA                                        1.64474


### Задание 2
Рассчитайте расстояние между всеми заведениями общепита из набора данных. Выведите первые 10.

In [11]:
light_rdd = data_rdd.map(lambda x: (x[0], x[1], x[12], x[13]))

all_pairs_rdd = light_rdd.cartesian(light_rdd).filter(lambda x: x[0][0] < x[1][0]).map(lambda x: (
    f"{x[0][1]} - {x[1][1]}",
    get_haversin_dist(x[0][2], x[0][3], x[1][2], x[1][3])
))

pairs_results = all_pairs_rdd.take(10)

print(f"{'Пара заведений':<65} {'Расстояние (км)':>10}")
print("-" * 75)
for pair, dist in pairs_results:
    pair_short = pair[:65] if len(pair) <= 65 else pair[:62] + "..."
    print(f"{pair_short:<65} {round(dist, 5):>10}")




Пара заведений                                                    Расстояние (км)
---------------------------------------------------------------------------
МУ-МУ - КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА                         0.68307
МУ-МУ - Флорентини                                                   1.52344
МУ-МУ - Макдоналдс                                                   2.62944
МУ-МУ - ЭЛЬ ГАУЧО                                                    2.05959
МУ-МУ - Темпл бар                                                    0.06241
МУ-МУ - Бейрут                                                       1.79928
МУ-МУ - Алтаргана                                                    1.71954
МУ-МУ - Византия                                                     1.73678
МУ-МУ - Unlock cafe                                                  3.34521
МУ-МУ - Брусника                                                     3.18203


### Задание 3
Выведите топ-10 наиболее близких и наиболее отдаленных заведений.


In [13]:
min_10 = target_dist_rdd.takeOrdered(10, key=lambda x: x[1])
max_10 = target_dist_rdd.takeOrdered(10, key=lambda x: -x[1])

print("\nТОП-10 БЛИЖАЙШИХ")
print("-" * 60)
print(f"{'Ранг':<5} {'Название':<45} {'Расстояние (км)':>10}")
print("-" * 60)
for rank, (name, dist) in enumerate(min_10, 1):
    name_short = name[:45] if len(name) <= 45 else name[:42] + "..."
    print(f"{rank:<5} {name_short:<45} {round(dist, 5):>10}")

print("\n\nТОП-10 САМЫХ ОТДАЛЕННЫХ")
print("-" * 60)
print(f"{'Ранг':<5} {'Название':<45} {'Расстояние (км)':>10}")
print("-" * 60)
for rank, (name, dist) in enumerate(max_10, 1):
    name_short = name[:45] if len(name) <= 45 else name[:42] + "..."
    print(f"{rank:<5} {name_short:<45} {round(dist, 5):>10}")
print("=" * 60 + "\n")



ТОП-10 БЛИЖАЙШИХ
------------------------------------------------------------
Ранг  Название                                      Расстояние (км)
------------------------------------------------------------
1     Мареа                                            1.05598
2     Стейк Хаус «Бизон»                                 1.059
3     ЛяМур                                            1.06532
4     БИБЛИОТЕКА Shisha Lounge                         1.06604
5     Му-Му                                            1.06604
6     Unlock cafe                                      1.06843
7     Папа Джонс                                       1.06847
8     Jimmy Poy                                        1.06891
9     Шоколадница                                       1.0699
10    Настоишная                                       1.06995


ТОП-10 САМЫХ ОТДАЛЕННЫХ
------------------------------------------------------------
Ранг  Название                                      Расстояние (км)
-------

### Статистика

In [15]:
distances_rdd = target_dist_rdd.map(lambda x: x[1])

min_dist = distances_rdd.min()
max_dist = distances_rdd.max()
avg_dist = distances_rdd.mean()
total = distances_rdd.count()

print(f"\nВсего заведений: {total}")
print(f"Минимум: {round(min_dist, 5)} км")
print(f"Максимум: {round(max_dist, 5)} км")
print(f"Среднее: {round(avg_dist, 5)} км")
print(f"Диапазон: {round(max_dist - min_dist, 5)} км")
print(f"Макс/Мин: {round(max_dist/min_dist, 2)}x")



Всего заведений: 768
Минимум: 1.05598 км
Максимум: 6.42686 км
Среднее: 2.75339 км
Диапазон: 5.37088 км
Макс/Мин: 6.09x


In [16]:
sc.stop()